In [1]:
# import pandas as pd
# import altair as alt
# alt.data_transformers.disable_max_rows()
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from csv import reader
from pyspark.sql.types import DoubleType,StringType,IntegerType,DateType
from pyspark.sql.functions import col,when,count,desc
from pyspark.sql.functions import format_string
import sys
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd

In [3]:
sc=SparkContext.getOrCreate()
spark = SparkSession(sc)
player = spark.read.format("csv").option("header", "true").load('/FileStore/tables//Player_T20Joined.csv')
delivery=spark.read.format("csv").option("header", "true").load('/FileStore/tables//deliveries_2019.csv')


In [4]:
player=player.withColumnRenamed('avg(Innings Batting Strike Rate)',"StrikeRate")
player=player.withColumnRenamed('avg(Innings Economy Rate)',"EconomyRate")
cols=["StrikeRate","EconomyRate"]
for col_name in cols:
    player = player.withColumn(col_name, col(col_name).cast('float'))

In [5]:
player.dtypes

Out[5]: [('Player_Name', 'string'),
 ('StrikeRate', 'float'),
 ('EconomyRate', 'float'),
 ('Player_Id', 'string'),
 ('DOB', 'string'),
 ('Batting_Hand', 'string'),
 ('Bowling_Skill', 'string'),
 ('Country', 'string')]

In [6]:
split_col = F.split(player['DOB'], '/')
player = player.withColumn('DOB', split_col.getItem(2))


player = player.withColumn("DOB", col("DOB").cast('float'))


player=player.withColumn("age",2020-player["DOB"])

In [7]:
mean_age=player.agg({'age': 'mean'})

# mean_age= 36.21311475409836
player=player.fillna( { 'age':36.21311475409836 } )

In [8]:
player = player.withColumn("age", col("age").cast('int'))
player.show()

+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+
 Player_Name|StrikeRate|EconomyRate|Player_Id| DOB|Batting_Hand| Bowling_Skill| Country|age|
+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+
 Kuldeep Yadav| 242.105| 7.2075| 462|1994.0| Left_Hand|Slow left-arm chi...| India| 26|
 TM Dilshan| 97.41257| 7.444| 135|1976.0| Right_Hand| Right-arm offbreak| Sri Lanka| 44|
 RMS Eranga| 85.71| 8.136666| null| null| null| null| null| 36|
Mohammad Mithun| 75.17728| null| null| null| null| null| null| 36|
 ST Finn| 65.55334| 7.3333335| null| null| null| null| null| 36|
 MS Crane| null| 7.75| null| null| null| null| null| 36|
 J Botha| 128.9675| 6.6953845| 192|1982.0| Right_Hand| Right-arm offbreak|South Africa| 38|
 J Charles| 100.87727| null| null| null| null| null| null| 36|
 KA Pollard| 126.55341| 8.997763| 221|1987.0| Right_Hand|Right-arm medium-...| West Indies| 33|
 M Muralitharan| 12.5| 6.3258333| 121|1972.0| Right_Hand| Right-arm offbreak| Sri Lanka| 48|
 KO Meth| 100.0| 9.25| null| null| null| null| null| 36|
 MA Ouma| 71.384735| null| null| null| null| null| null| 36|
 T Lungameni| 100.0| 6.2066665| null| null| null| null| null| 36|
 AR White| 102.11| 4.5| null| null| null| null| null| 36|
 JN Malan| 69.89| null| null| null| null| null| null| 36|
 SS Cottrell| 60.88286| 8.013798| null| null| null| null| null| 36|
 T Panyangara| 88.568| 7.835| null| null| null| null| null| 36|
 RJ Kirtley| 50.0| 17.0| null| null| null| null| null| 36|
 CA Ingram| 99.50889| null| 292|1985.0| Left_Hand| Legbreak|South Africa| 35|
 GL Cloete| 87.495| null| null| null| null| null| null| 36|
+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+
only showing top 20 rows

In [9]:
player=player.fillna( { 'StrikeRate':0 ,'EconomyRate':0} )
player=player.withColumn("label",F.when((F.col("StrikeRate") ==0.0),"Bowler")\
                         .when((F.col("EconomyRate") ==0.0),"Batsman")\
                         .when((F.col("EconomyRate") !=0) & (F.col("StrikeRate") !=0),"AllRounder"))

In [10]:
player.show()

+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+----------+
 Player_Name|StrikeRate|EconomyRate|Player_Id| DOB|Batting_Hand| Bowling_Skill| Country|age| label|
+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+----------+
 Kuldeep Yadav| 242.105| 7.2075| 462|1994.0| Left_Hand|Slow left-arm chi...| India| 26|AllRounder|
 TM Dilshan| 97.41257| 7.444| 135|1976.0| Right_Hand| Right-arm offbreak| Sri Lanka| 44|AllRounder|
 RMS Eranga| 85.71| 8.136666| null| null| null| null| null| 36|AllRounder|
Mohammad Mithun| 75.17728| 0.0| null| null| null| null| null| 36| Batsman|
 ST Finn| 65.55334| 7.3333335| null| null| null| null| null| 36|AllRounder|
 MS Crane| 0.0| 7.75| null| null| null| null| null| 36| Bowler|
 J Botha| 128.9675| 6.6953845| 192|1982.0| Right_Hand| Right-arm offbreak|South Africa| 38|AllRounder|
 J Charles| 100.87727| 0.0| null| null| null| null| null| 36| Batsman|
 KA Pollard| 126.55341| 8.997763| 221|1987.0| Right_Hand|Right-arm medium-...| West Indies| 33|AllRounder|
 M Muralitharan| 12.5| 6.3258333| 121|1972.0| Right_Hand| Right-arm offbreak| Sri Lanka| 48|AllRounder|
 KO Meth| 100.0| 9.25| null| null| null| null| null| 36|AllRounder|
 MA Ouma| 71.384735| 0.0| null| null| null| null| null| 36| Batsman|
 T Lungameni| 100.0| 6.2066665| null| null| null| null| null| 36|AllRounder|
 AR White| 102.11| 4.5| null| null| null| null| null| 36|AllRounder|
 JN Malan| 69.89| 0.0| null| null| null| null| null| 36| Batsman|
 SS Cottrell| 60.88286| 8.013798| null| null| null| null| null| 36|AllRounder|
 T Panyangara| 88.568| 7.835| null| null| null| null| null| 36|AllRounder|
 RJ Kirtley| 50.0| 17.0| null| null| null| null| null| 36|AllRounder|
 CA Ingram| 99.50889| 0.0| 292|1985.0| Left_Hand| Legbreak|South Africa| 35| Batsman|
 GL Cloete| 87.495| 0.0| null| null| null| null| null| 36| Batsman|
+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+----------+
only showing top 20 rows

In [11]:
delivery.dtypes
cols=["total_runs"]
for col_name in cols:
    delivery = delivery.withColumn(col_name, col(col_name).cast('float'))
bowler_data=delivery.filter(delivery["player_dismissed"]!=" ")

In [12]:
bowler_data=bowler_data.select("bowler")
bowler=bowler_data.groupby("bowler").count()

In [13]:
bowler=bowler.withColumnRenamed("count","wickets")
bowler=bowler.withColumnRenamed("bowler","Player_Name")
bowler.show()

+---------------+-------+
 Player_Name|wickets|
+---------------+-------+
 Kuldeep Yadav| 42|
 TM Dilshan| 5|
 A Roy| 1|
 M Muralitharan| 67|
 LA Carseldine| 1|
 J Botha| 27|
 KA Pollard| 67|
 DR Smith| 27|
 Jaskaran Singh| 6|
 A Flintoff| 2|
 GR Napier| 1|
 S Warrier| 2|
 AR Patel| 76|
 B Lee| 30|
 D du Preez| 4|
 BMAJ Mendis| 1|
 O Thomas| 5|
NM Coulter-Nile| 39|
 HV Patel| 47|
Mohammad Hafeez| 2|
+---------------+-------+
only showing top 20 rows

In [14]:
inner_join = player.join(bowler, on=["Player_Name"],how="left")
inner_join.show()

+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+----------+-------+
 Player_Name|StrikeRate|EconomyRate|Player_Id| DOB|Batting_Hand| Bowling_Skill| Country|age| label|wickets|
+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+----------+-------+
 Kuldeep Yadav| 242.105| 7.2075| 462|1994.0| Left_Hand|Slow left-arm chi...| India| 26|AllRounder| 42|
 TM Dilshan| 97.41257| 7.444| 135|1976.0| Right_Hand| Right-arm offbreak| Sri Lanka| 44|AllRounder| 5|
 RMS Eranga| 85.71| 8.136666| null| null| null| null| null| 36|AllRounder| null|
Mohammad Mithun| 75.17728| 0.0| null| null| null| null| null| 36| Batsman| null|
 ST Finn| 65.55334| 7.3333335| null| null| null| null| null| 36|AllRounder| null|
 MS Crane| 0.0| 7.75| null| null| null| null| null| 36| Bowler| null|
 J Botha| 128.9675| 6.6953845| 192|1982.0| Right_Hand| Right-arm offbreak|South Africa| 38|AllRounder| 27|
 J Charles| 100.87727| 0.0| null| null| null| null| null| 36| Batsman| null|
 KA Pollard| 126.55341| 8.997763| 221|1987.0| Right_Hand|Right-arm medium-...| West Indies| 33|AllRounder| 67|
 M Muralitharan| 12.5| 6.3258333| 121|1972.0| Right_Hand| Right-arm offbreak| Sri Lanka| 48|AllRounder| 67|
 KO Meth| 100.0| 9.25| null| null| null| null| null| 36|AllRounder| null|
 MA Ouma| 71.384735| 0.0| null| null| null| null| null| 36| Batsman| null|
 T Lungameni| 100.0| 6.2066665| null| null| null| null| null| 36|AllRounder| null|
 AR White| 102.11| 4.5| null| null| null| null| null| 36|AllRounder| null|
 JN Malan| 69.89| 0.0| null| null| null| null| null| 36| Batsman| null|
 SS Cottrell| 60.88286| 8.013798| null| null| null| null| null| 36|AllRounder| null|
 T Panyangara| 88.568| 7.835| null| null| null| null| null| 36|AllRounder| null|
 RJ Kirtley| 50.0| 17.0| null| null| null| null| null| 36|AllRounder| null|
 CA Ingram| 99.50889| 0.0| 292|1985.0| Left_Hand| Legbreak|South Africa| 35| Batsman| null|
 GL Cloete| 87.495| 0.0| null| null| null| null| null| 36| Batsman| null|
+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+----------+-------+
only showing top 20 rows

In [15]:
batsman_data=delivery.groupby("batsman").count()
batsman_data.columns
batsman_data=batsman_data.withColumnRenamed("batsman","Player_Name")
batsman_data=batsman_data.withColumnRenamed("count","runs")
# batsman_data=batsman_data.filter("")

In [17]:
inner_join=inner_join.join(batsman_data, on=["Player_Name"],how="left")

In [18]:
inner_join=inner_join.fillna( { 'wickets':0 ,'runs':0} )
inner_join.show()

+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+----------+-------+----+
 Player_Name|StrikeRate|EconomyRate|Player_Id| DOB|Batting_Hand| Bowling_Skill| Country|age| label|wickets|runs|
+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+----------+-------+----+
 Kuldeep Yadav| 242.105| 7.2075| 462|1994.0| Left_Hand|Slow left-arm chi...| India| 26|AllRounder| 42| 55|
 TM Dilshan| 97.41257| 7.444| 135|1976.0| Right_Hand| Right-arm offbreak| Sri Lanka| 44|AllRounder| 5|1047|
 RMS Eranga| 85.71| 8.136666| null| null| null| null| null| 36|AllRounder| 0| 0|
Mohammad Mithun| 75.17728| 0.0| null| null| null| null| null| 36| Batsman| 0| 0|
 ST Finn| 65.55334| 7.3333335| null| null| null| null| null| 36|AllRounder| 0| 0|
 MS Crane| 0.0| 7.75| null| null| null| null| null| 36| Bowler| 0| 0|
 J Botha| 128.9675| 6.6953845| 192|1982.0| Right_Hand| Right-arm offbreak|South Africa| 38|AllRounder| 27| 364|
 J Charles| 100.87727| 0.0| null| null| null| null| null| 36| Batsman| 0| 0|
 KA Pollard| 126.55341| 8.997763| 221|1987.0| Right_Hand|Right-arm medium-...| West Indies| 33|AllRounder| 67|1964|
 M Muralitharan| 12.5| 6.3258333| 121|1972.0| Right_Hand| Right-arm offbreak| Sri Lanka| 48|AllRounder| 67| 30|
 KO Meth| 100.0| 9.25| null| null| null| null| null| 36|AllRounder| 0| 0|
 MA Ouma| 71.384735| 0.0| null| null| null| null| null| 36| Batsman| 0| 0|
 T Lungameni| 100.0| 6.2066665| null| null| null| null| null| 36|AllRounder| 0| 0|
 AR White| 102.11| 4.5| null| null| null| null| null| 36|AllRounder| 0| 0|
 JN Malan| 69.89| 0.0| null| null| null| null| null| 36| Batsman| 0| 0|
 SS Cottrell| 60.88286| 8.013798| null| null| null| null| null| 36|AllRounder| 0| 0|
 T Panyangara| 88.568| 7.835| null| null| null| null| null| 36|AllRounder| 0| 0|
 RJ Kirtley| 50.0| 17.0| null| null| null| null| null| 36|AllRounder| 0| 0|
 CA Ingram| 99.50889| 0.0| 292|1985.0| Left_Hand| Legbreak|South Africa| 35| Batsman| 0| 26|
 GL Cloete| 87.495| 0.0| null| null| null| null| null| 36| Batsman| 0| 0|
+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+----------+-------+----+
only showing top 20 rows

In [19]:
inner_join.filter(inner_join["Player_Name"]=="V Kohli").show()

+-----------+----------+-----------+---------+------+------------+----------------+-------+---+----------+-------+----+
Player_Name|StrikeRate|EconomyRate|Player_Id| DOB|Batting_Hand| Bowling_Skill|Country|age| label|wickets|runs|
+-----------+----------+-----------+---------+------+------------+----------------+-------+---+----------+-------+----+
 V Kohli| 127.14158| 8.835834| 8|1988.0| Right_Hand|Right-arm medium| India| 32|AllRounder| 5|4211|
+-----------+----------+-----------+---------+------+------------+----------------+-------+---+----------+-------+----+

In [20]:
inner_join=inner_join.withColumn("bat_val",F.when((F.col("runs") <800),2)\
                         .when((F.col("runs") >= 800)& (F.col("runs") < 1600),3)\
                         .when((F.col("runs") >= 1600)& (F.col("runs") < 2400),4)\
                         .when((F.col("runs") >= 2400)& (F.col("runs") < 3200),5)\
                         .when((F.col("runs") >= 3200)& (F.col("runs") < 3600),6)\
                         .when((F.col("runs") >= 3600)& (F.col("runs") < 4000),7)\
                         .when((F.col("runs") >= 4000)& (F.col("runs") < 4400),8)\
                         .when((F.col("runs") >= 4400)& (F.col("runs") < 14800),9)\
                         .when((F.col("runs") >= 4800),10))

In [21]:
inner_join=inner_join.withColumn("bowl_val",F.when((F.col("wickets") <15),2)\
                         .when((F.col("wickets") >= 15)& (F.col("wickets") < 25),3)\
                         .when((F.col("wickets") >= 25)& (F.col("wickets") < 35),4)\
                         .when((F.col("wickets") >= 35)& (F.col("wickets") < 55),5)\
                         .when((F.col("wickets") >= 55)& (F.col("wickets") < 80),6)\
                         .when((F.col("wickets") >= 80)& (F.col("wickets") < 105),7)\
                         .when((F.col("wickets") >= 105)& (F.col("wickets") < 130),8)\
                         .when((F.col("wickets") >= 130)& (F.col("wickets") < 150),9)\
                         .when((F.col("wickets") >= 150),10))
inner_join=inner_join.withColumn("age_val",F.when((F.col("age") >=40),2)\
                         .when((F.col("age") >= 35)& (F.col("age") < 40),3)\
                         .when((F.col("age") >= 32)& (F.col("age") < 35),4)\
                         .when((F.col("age") >= 30)& (F.col("age") < 32),5)\
                         .when((F.col("age") >= 28)& (F.col("age") < 30),6)\
                         .when((F.col("age") >= 26)& (F.col("age") < 28),7)\
                         .when((F.col("age") >= 24)& (F.col("age") < 26),8)\
                         .when((F.col("age") >= 22)& (F.col("age") < 24),9)\
                         .when((F.col("age") <= 22),10))

In [22]:
inner_join=inner_join.withColumn("StrikeRate_val",10*inner_join["StrikeRate"]/325)
inner_join=inner_join.withColumn("EconomyRate_val",10*inner_join["EconomyRate"]/24)

In [23]:
inner_join.show()

+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+----------+-------+----+-------+--------+-------+-------------------+------------------+
 Player_Name|StrikeRate|EconomyRate|Player_Id| DOB|Batting_Hand| Bowling_Skill| Country|age| label|wickets|runs|bat_val|bowl_val|age_val| StrikeRate_val| EconomyRate_val|
+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+----------+-------+----+-------+--------+-------+-------------------+------------------+
 Kuldeep Yadav| 242.105| 7.2075| 462|1994.0| Left_Hand|Slow left-arm chi...| India| 26|AllRounder| 42| 55| 2| 5| 7| 7.449384765625|3.0031248728434243|
 TM Dilshan| 97.41257| 7.444| 135|1976.0| Right_Hand| Right-arm offbreak| Sri Lanka| 44|AllRounder| 5|1047| 3| 2| 2| 2.997309758112981|3.1016664505004883|
 RMS Eranga| 85.71| 8.136666| null| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 2.6372306941105768| 3.390277544657389|
Mohammad Mithun| 75.17728| 0.0| null| null| null| null| null| 36| Batsman| 0| 0| 2| 2| 3| 2.31314697265625| 0.0|
 ST Finn| 65.55334| 7.3333335| null| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 2.0170258037860576|3.0555556615193686|
 MS Crane| 0.0| 7.75| null| null| null| null| null| 36| Bowler| 0| 0| 2| 2| 3| 0.0|3.2291666666666665|
 J Botha| 128.9675| 6.6953845| 192|1982.0| Right_Hand| Right-arm offbreak|South Africa| 38|AllRounder| 27| 364| 2| 4| 3| 3.968230919471154| 2.789743423461914|
 J Charles| 100.87727| 0.0| null| null| null| null| null| 36| Batsman| 0| 0| 2| 2| 3| 3.103916015625| 0.0|
 KA Pollard| 126.55341| 8.997763| 221|1987.0| Right_Hand|Right-arm medium-...| West Indies| 33|AllRounder| 67|1964| 4| 6| 4| 3.8939513221153845| 3.749067942301432|
 M Muralitharan| 12.5| 6.3258333| 121|1972.0| Right_Hand| Right-arm offbreak| Sri Lanka| 48|AllRounder| 67| 30| 2| 6| 2|0.38461538461538464|2.6357638041178384|
 KO Meth| 100.0| 9.25| null| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 3.076923076923077|3.8541666666666665|
 MA Ouma| 71.384735| 0.0| null| null| null| null| null| 36| Batsman| 0| 0| 2| 2| 3| 2.196453387920673| 0.0|
 T Lungameni| 100.0| 6.2066665| null| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 3.076923076923077| 2.586111068725586|
 AR White| 102.11| 4.5| null| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 3.1418460787259614| 1.875|
 JN Malan| 69.89| 0.0| null| null| null| null| null| 36| Batsman| 0| 0| 2| 2| 3| 2.1504616135817307| 0.0|
 SS Cottrell| 60.88286| 8.013798| null| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 1.8733188100961538| 3.339082400004069|
 T Panyangara| 88.568| 7.835| null| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 2.725169208233173|3.2645832697550454|
 RJ Kirtley| 50.0| 17.0| null| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 1.5384615384615385| 7.083333333333333|
 CA Ingram| 99.50889| 0.0| 292|1985.0| Left_Hand| Legbreak|South Africa| 35| Batsman| 0| 26| 2| 2| 3| 3.0618118990384615| 0.0|
 GL Cloete| 87.495| 0.0| null| null| null| null| null| 36| Batsman| 0| 0| 2| 2| 3| 2.6921538837139423| 0.0|
+---------------+----------+-----------+---------+------+------------+--------------------+------------+---+----------+-------+----+-------+--------+-------+-------------------+------------------+
only showing top 20 rows

In [24]:
inner_join=inner_join.withColumn("Rating",(inner_join["age_val"]+inner_join["bat_val"]+inner_join["bowl_val"]+inner_join["StrikeRate_val"]+inner_join["EconomyRate_val"])/5)

In [25]:
minrating=inner_join.agg({"Rating":"min"}).collect()[0]
for i in minrating:
  minR=i

maxrating=inner_join.agg({"Rating":"max"}).collect()[0]
for i in maxrating:
  maxR=i

inner_join=inner_join.withColumn('RatingNorm', (inner_join.Rating-minR)*10/(maxR-minR))
      
inner_join=inner_join.withColumn("Buy_Type",F.when((F.col("RatingNorm") <2.5),"Avoid")\
                         .when((F.col("RatingNorm") >= 2.5)& (F.col("RatingNorm") < 5.0),"Maybe")\
                         .when((F.col("RatingNorm") >= 5.0)& (F.col("RatingNorm") < 7.5),"Good Buy")\
                         .when((F.col("RatingNorm") >=7.5),"Excellent"))


In [26]:
inner_join=inner_join.drop("Country")
inner_join.show()

+---------------+----------+-----------+---------+------+------------+--------------------+---+----------+-------+----+-------+--------+-------+-------------------+------------------+------------------+------------------+---------+
 Player_Name|StrikeRate|EconomyRate|Player_Id| DOB|Batting_Hand| Bowling_Skill|age| label|wickets|runs|bat_val|bowl_val|age_val| StrikeRate_val| EconomyRate_val| Rating| RatingNorm| Buy_Type|
+---------------+----------+-----------+---------+------+------------+--------------------+---+----------+-------+----+-------+--------+-------+-------------------+------------------+------------------+------------------+---------+
 Kuldeep Yadav| 242.105| 7.2075| 462|1994.0| Left_Hand|Slow left-arm chi...| 26|AllRounder| 42| 55| 2| 5| 7| 7.449384765625|3.0031248728434243| 4.890501927693685| 10.0|Excellent|
 TM Dilshan| 97.41257| 7.444| 135|1976.0| Right_Hand| Right-arm offbreak| 44|AllRounder| 5|1047| 3| 2| 2| 2.997309758112981|3.1016664505004883|2.6197952417226937|3.4946127147067974| Maybe|
 RMS Eranga| 85.71| 8.136666| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 2.6372306941105768| 3.390277544657389|2.6055016477535933|3.4536627474379213| Maybe|
Mohammad Mithun| 75.17728| 0.0| null| null| null| null| 36| Batsman| 0| 0| 2| 2| 3| 2.31314697265625| 0.0| 1.86262939453125|1.3253950409273314| Avoid|
 ST Finn| 65.55334| 7.3333335| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 2.0170258037860576|3.0555556615193686|2.4145162930610855|2.9065054656234417| Maybe|
 MS Crane| 0.0| 7.75| null| null| null| null| 36| Bowler| 0| 0| 2| 2| 3| 0.0|3.2291666666666665|2.0458333333333334|1.8502592083083635| Avoid|
 J Botha| 128.9675| 6.6953845| 192|1982.0| Right_Hand| Right-arm offbreak| 38|AllRounder| 27| 364| 2| 4| 3| 3.968230919471154| 2.789743423461914|3.1515948685866135| 5.018174763604737| Good Buy|
 J Charles| 100.87727| 0.0| null| null| null| null| 36| Batsman| 0| 0| 2| 2| 3| 3.103916015625| 0.0| 2.020783203125| 1.778492652302234| Avoid|
 KA Pollard| 126.55341| 8.997763| 221|1987.0| Right_Hand|Right-arm medium-...| 33|AllRounder| 67|1964| 4| 6| 4| 3.8939513221153845| 3.749067942301432| 4.328603852883363| 8.390208381344197|Excellent|
 M Muralitharan| 12.5| 6.3258333| 121|1972.0| Right_Hand| Right-arm offbreak| 48|AllRounder| 67| 30| 2| 6| 2|0.38461538461538464|2.6357638041178384|2.6040758377466444| 3.449577919420391| Maybe|
 KO Meth| 100.0| 9.25| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 3.076923076923077|3.8541666666666665|2.7862179487179484|3.9714000376842034| Maybe|
 MA Ouma| 71.384735| 0.0| null| null| null| null| 36| Batsman| 0| 0| 2| 2| 3| 2.196453387920673| 0.0|1.8392906775841347|1.2585315426953274| Avoid|
 T Lungameni| 100.0| 6.2066665| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 3.076923076923077| 2.586111068725586|2.5326068291297323| 3.244825107081638| Maybe|
 AR White| 102.11| 4.5| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 3.1418460787259614| 1.875|2.4033692157451925|2.8745700089275097| Maybe|
 JN Malan| 69.89| 0.0| null| null| null| null| 36| Batsman| 0| 0| 2| 2| 3| 2.1504616135817307| 0.0|1.8300923227163461|1.2321790150121061| Avoid|
 SS Cottrell| 60.88286| 8.013798| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 1.8733188100961538| 3.339082400004069| 2.442480242020044| 2.986619871912957| Maybe|
 T Panyangara| 88.568| 7.835| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 2.725169208233173|3.2645832697550454|2.5979504955976434|3.4320293196032625| Maybe|
 RJ Kirtley| 50.0| 17.0| null| null| null| null| 36|AllRounder| 0| 0| 2| 2| 3| 1.5384615384615385| 7.083333333333333|3.1243589743589744| 4.940146174044166| Maybe|
 CA Ingram| 99.50889| 0.0| 292|1985.0| Left_Hand| Legbreak| 35| Batsman| 0| 26| 2| 2| 3| 3.0618118990384615| 0.0|2.0123623798076924|1.7543676883522161| Avoid|
 GL Cloete| 87.495| 0.0| null| null| null| null| 36| Batsman| 0| 0| 2| 2| 3| 2.6921538837139423| 0.0|1.9384307767427884|1.5425597461238227| Avoid|
+---------------+----------+---------

In [27]:
x=inner_join.filter(inner_join["Buy_Type"]=="Excellent")
display(x.select("Player_Name"))

Player_Name
Kuldeep Yadav
KA Pollard
AR Patel
RG Sharma
B Kumar
R Ashwin
KH Pandya
SP Narine
HH Pandya
V Sehwag


In [28]:
cols=["StrikeRate","EconomyRate","DOB"]
for col_name in cols:
    inner_join = inner_join.withColumn(col_name, col(col_name).cast('double'))
inner_join.dtypes

Out[48]: [('Player_Name', 'string'),
 ('StrikeRate', 'double'),
 ('EconomyRate', 'double'),
 ('Player_Id', 'string'),
 ('DOB', 'double'),
 ('Batting_Hand', 'string'),
 ('Bowling_Skill', 'string'),
 ('age', 'int'),
 ('label', 'string'),
 ('wickets', 'bigint'),
 ('runs', 'bigint'),
 ('bat_val', 'int'),
 ('bowl_val', 'int'),
 ('age_val', 'int'),
 ('StrikeRate_val', 'double'),
 ('EconomyRate_val', 'double'),
 ('Rating', 'double'),
 ('RatingNorm', 'double'),
 ('Buy_Type', 'string')]

In [29]:
# inner_join_other=inner_join.select(["Player_Name",])
inner_join=inner_join.select("Player_Name",'StrikeRate','EconomyRate','Batting_Hand','Bowling_Skill','age','label','wickets','runs',"RatingNorm","Buy_Type")


In [30]:
inner_join.write.format("com.mongodb.spark.sql.DefaultSource").mode("overwrite").option("database", "IPL").option("collection", "ratings").save()
# inner_join_other.write.format("com.mongodb.spark.sql.DefaultSource").mode("overwrite").option("database", "IPL").option("collection", "fields").save()